In [ ]:
# import libraries
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import math
import win32com.client, time
from queue import Queue
import pyautogui
import mss 
from helper_file import *

In [3]:
# start game
shell = win32com.client.Dispatch("WScript.Shell")
# if shell.Activate('Play games, WIN REAL REWARDS! | GAMEE') == False:
result =shell.Popup("Would you like to run Karate Kido2", 0, "Karate Kido2 Launcher",35)
if result == 6:
    shell.Run('https://prizes.gamee.com/game/karatekid2')
    shell.Popup("Running game...", 4, "Wait!")
    time.sleep(2)
    
    shell.Popup("Hover your Mouse over the PLAY button location in the next 3 seconds...", 4, "Instruction!")
    time.sleep(1)

    play_button = pyautogui.position()
    play_button_x = play_button[0]
    play_button_y = play_button[1]

    shell.Popup("Hover your Mouse over the LEFT position in the next 3 seconds...", 4, "Instruction!")
    time.sleep(1)

    left_button = pyautogui.position()
    left_button_x = left_button[0]
    left_button_y = left_button[1]

    shell.Popup("Hover your Mouse over the RIGHT position in the next 3 seconds...", 4, "Instruction!")
    time.sleep(1)
    right_button = pyautogui.position()
    right_button_x = right_button[0]
    right_button_y = right_button[1]
    
    shell.Popup("All Done, the game will start playing \n   HAVE FUN", 4, "Done!")
    
else:
    shell.Popup("Maybe next time.", 4, "Bye!")
    
    
    


In [ ]:
# click to start
pyautogui.click(x=play_button_x, y=play_button_y)
time.sleep(0.2)
pyautogui.click(x=play_button_x, y=play_button_y)
time.sleep(0.4)

In [65]:
# detect window
if result == 6:
    with mss.mss() as sct: 
        screen_shot = sct.shot(output="result.png") 
    #  1.8, 3.5, 1.7, 2.2, 2.5
    full_screenshot = cv.imread('result.png')
    game_window_template = cv.imread('game_window.png')
    
    window_top_left, window_bottom_right = detect_game_window(game_window_template, full_screenshot)


In [66]:
# 0.3, 0.2, 0.3, 0.3, 0.3

top = window_top_left[1]
left = window_top_left[0]
width = window_bottom_right[0] - window_top_left[0] 
height = window_bottom_right[1] - window_top_left[1]

image = full_screenshot[top : top + height, left : left + width]
# image = cv.imread('screen-shots/12.png')

# preprocess (add dark blue for horizontal)
vertical_lines, horizontal_lines = preprocess_log(image)

log_center, log_top_left, log_bottom_right = log_coordinate(image, vertical_lines)

# player 
player_center, player_top_left, player_bottom_right = match_characters(image)
player_center = (230, 650)

# player
player_placement = None
player_log_middle_distance = player_center[0] - log_center[0]
if player_log_middle_distance > 0:
    player_placement = 'right'
else:
    player_placement = 'left'
   
# copy = image 
# cv.rectangle(copy, log_top_left, log_bottom_right, (255, 0, 0), 2) 
# cv.rectangle(copy, player_top_left, player_bottom_right, (0, 255, 0), 2)
# cv.circle(copy, player_center, 2, (0, 255, 255), 20)
# plt.imshow(copy)
# plt.show()


In [ ]:
with mss.mss() as sct:
   
    monitor = {"top": int(top), "left": int(left), "width": int(width), "height": int(height)}

    print('start')  
    i = 0  
    while "Screen capturing":
        
        frame = np.array(sct.grab(monitor))
        
        vertical_lines, horizontal_lines= preprocess_log(frame)
        # 40, 15
        branches = branches_coordinates(log_center, (log_top_left[0] - 10, log_top_left[1]), (log_bottom_right[0] + 10, log_bottom_right[1]), horizontal_lines)
        
        glass_center, glass_top_left, glass_bottom_right = glass_filter(log_bottom_right, frame)
        
        number_value, number_center, number_top_left, number_bottom_right = determine_number(frame)
        
        power_up_center = power_ups_filter(frame)
        if power_up_center == None:
            power_up_center = (0, 0)
        _, branches_positions, power_up_placement = decision_making(player_center, log_center, branches, power_up_center)     

        # movement(player_center, player_placement, power_up_center, power_up_placement, branches_positions, glass_center, number_value, number_center, right_button_x, right_button_y, left_button_x, left_button_y)
        
        # make decision
        branch = None
        for element in range(len(branches_positions) - 1, 0, -1):
            branch = branches_positions[element]
            if branch['position'] == player_placement:
                break
       
        
        if abs(player_center[1] - power_up_center[1]) < 150:
            print('catch power up')
            
        if player_placement == 'right' and power_up_placement != player_placement and abs(player_center[1] - power_up_center[1]) < 180:
            player_placement = 'left'        
        elif player_placement == 'left' and power_up_placement != player_placement and abs(player_center[1] - power_up_center[1]) < 180:
            player_placement = 'right'
        
         # 180, 150
        if player_placement == 'right' and abs(player_center[1] - branch['coordinate']) < 150:
            player_placement = 'left'
        elif player_placement == 'left' and abs(player_center[1] - branch['coordinate']) < 150:
            player_placement = 'right'
            
        move_x = None
        move_y = None
        if player_placement == 'right':
            move_x = right_button_x
            move_y = right_button_y
        else:
            move_x = left_button_x
            move_y = left_button_y


        if abs(glass_center[1] - player_center[1]) < 180 and abs(glass_center[1] - player_center[1]) > 40:
                print(f'frame{i}: hit glass')
                pyautogui.click(x=move_x, y=move_y)

        if abs(number_center[1] - player_center[1]) < 95:
            print(f'frame{i}: hit {number_value}')
            for counter in range(1, number_value):
                pyautogui.click(x=move_x, y=move_y)                
            # pyautogui.click(x=move_x, y=move_y)                


        pyautogui.click(x=move_x, y=move_y)
        
        frame = draw(frame, player_center, log_top_left, log_bottom_right, power_up_center, glass_top_left, glass_bottom_right, number_top_left, number_bottom_right, branches)
        
        cv.circle(frame, player_center, 2, (255, 255, 255), 20)
        # store
        cv.imwrite(f'result_screening{i}.png', frame)
        
        i = i + 1
